In [48]:
import pandas as pd
import numpy as np
import math
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [49]:
# Carregar dataset
data = pd.read_csv(r"ml-100k\u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])
data = data.drop('timestamp', axis=1)
data.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [50]:
# Divisão dos dados
"""""
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(f"Tamanho do conjunto do dataset: {len(data)}")
print(f"Tamanho do conjunto de teste: {len(test_data)}")
print(f"Tamanho do conjunto de treino: {len(train_data)}")
"""""

'""\ntrain_data, test_data = train_test_split(data, test_size=0.2, random_state=42)\nprint(f"Tamanho do conjunto do dataset: {len(data)}")\nprint(f"Tamanho do conjunto de teste: {len(test_data)}")\nprint(f"Tamanho do conjunto de treino: {len(train_data)}")\n'

In [51]:
class Avaliacao:
    def __init__(self, usuario_id, filme_id, avaliacao):
        self.usuario = usuario_id - 1
        self.filme = filme_id - 1
        self.avaliacao = avaliacao
        
# Definição da classe MatrizSVD
"""
parametros:
regularizador = lambda
k = 6
taxa_aprendizado = l_rate = 0.0066
rank = variaveis latentes
"""
class MatrizSVD:
    def __init__(self, dados_treino, num_usuarios, num_filmes, regularizador, rank= 30, taxa_aprendizado=0.066, tipo_arquivo=0):
        self.avaliacoes_treino = []
        self.avaliacoes_teste = []
        self.num_usuarios = num_usuarios
        self.num_filmes = num_filmes

        if tipo_arquivo == 0:
            self.ler_treino(dados_treino)
        elif tipo_arquivo == 1:
            self.ler_treino_grande(dados_treino)

        media = self.media_avaliacoes()
        valor_inicial = math.sqrt(media / rank)
        self.U = [[valor_inicial] * rank for _ in range(num_usuarios)]
        self.V = [[valor_inicial] * rank for _ in range(num_filmes)]
        self.rank = rank
        self.taxa_aprendizado = taxa_aprendizado
        self.regularizador = regularizador
        self.minimo_melhoria = 0.001
        self.max_epocas = 30

    def produto_escalar(self, v1, v2):
        return sum([v1[i] * v2[i] for i in range(len(v1))])

    def calcular_avaliacao(self, usuario_id, filme_id):
        p = self.produto_escalar(self.U[usuario_id], self.V[filme_id])
        return max(1, min(p, 5))

    def media_avaliacoes(self):
        soma = sum(av.avaliacao for av in self.avaliacoes_treino)
        return soma / len(self.avaliacoes_treino)

    def prever(self, usuario_id, filme_id):
        return self.calcular_avaliacao(usuario_id, filme_id)

    def treinar(self, k):
        erro_quadratico_soma = 0
        for av in self.avaliacoes_treino:
            erro = av.avaliacao - self.prever(av.usuario, av.filme)
            erro_quadratico_soma += erro ** 2
            u_temp = self.U[av.usuario][k]
            v_temp = self.V[av.filme][k]
            self.U[av.usuario][k] += self.taxa_aprendizado * (erro * v_temp - self.regularizador * u_temp)
            self.V[av.filme][k] += self.taxa_aprendizado * (erro * u_temp - self.regularizador * v_temp)
        return math.sqrt(erro_quadratico_soma / len(self.avaliacoes_treino))

    def treinar_avaliacoes(self):
        erro_treino_antigo = 1e6
        for k in range(self.rank):
            for epoca in range(self.max_epocas):
                erro_treino = self.treinar(k)
                if abs(erro_treino_antigo - erro_treino) < self.minimo_melhoria:
                    break
                erro_treino_antigo = erro_treino

    def calcular_rmse(self, avaliacoes):
        erro_quadratico_soma = sum((av.avaliacao - self.calcular_avaliacao(av.usuario, av.filme)) ** 2 for av in avaliacoes)
        return math.sqrt(erro_quadratico_soma / len(avaliacoes))

    def calcular_mae(self, avaliacoes):
        erro_absoluto_soma = sum(abs(av.avaliacao - self.calcular_avaliacao(av.usuario, av.filme)) for av in avaliacoes)
        return erro_absoluto_soma / len(avaliacoes)

    def ler_avaliacoes(self, nome_arquivo, avaliacoes, delimitador="\t"):
        with open(nome_arquivo) as arquivo:
            for linha in arquivo:
                usuario_id, filme_id, avaliacao = map(int, linha.split(delimitador)[:3])
                avaliacoes.append(Avaliacao(usuario_id, filme_id, avaliacao))
        avaliacoes.sort(key=lambda av: (av.usuario, av.filme))

    def ler_treino(self, dados_treino):
        for row in dados_treino.itertuples():
            self.avaliacoes_treino.append(Avaliacao(row.user_id, row.item_id, row.rating))

    def ler_teste(self, dados_teste):
        for row in dados_teste.itertuples():
            self.avaliacoes_teste.append(Avaliacao(row.user_id, row.item_id, row.rating))



In [52]:
num = 6
kf = KFold(n_splits=num)
mae_scores = []
rmse_scores = []
i = 0
for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]
    i = i + 1
    svd = MatrizSVD(train_data, data['user_id'].nunique(), data['item_id'].nunique(),0.01)
    svd.treinar_avaliacoes()
    svd.ler_teste(test_data)
    mae = svd.calcular_mae(svd.avaliacoes_teste)
    rmse = svd.calcular_rmse(svd.avaliacoes_teste)
    #print("k =", i)
    #print("MAE =", mae)
    #print("RMSE = ",rmse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
print("k =",num)
print("Média MAE:", np.mean(mae_scores))
print("Média RMSE:", np.mean(rmse_scores))

k = 6
Média MAE: 0.7350038132564191
Média RMSE: 0.9391842727025145


In [58]:
num = 20
kf = KFold(n_splits=num)
mae_scores = []
rmse_scores = []
refmae = []
refrmse = []
insane = []
for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]
    svd = MatrizSVD(train_data, data['user_id'].nunique(), data['item_id'].nunique(),0.01)
    svd.treinar_avaliacoes()
    svd.ler_teste(test_data)
    mae = svd.calcular_mae(svd.avaliacoes_teste)
    rmse = svd.calcular_rmse(svd.avaliacoes_teste)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
print ("Lambda =", i)
print("Média MAE:", np.mean(mae_scores))
print("Média RMSE:", np.mean(rmse_scores))
refmae.append(np.mean(mae_scores))
refrmse.append(np.mean(rmse_scores))
insane.append(i)
print("k =",num)


Lambda = 0.04
Média MAE: 0.7285308732199356
Média RMSE: 0.9307947816532041
k = 20
